In [3]:
import os
import pandas as pd

In [4]:

os.chdir('/project/yangili1/cdai/splice-pub/analysis')

In [5]:
sqtl_f = [f'../code/results/qtl/noisy/GTEx/Liver/separateNoise/cis_100000/perm/chr{x}.addQval.txt.gz' for x in range(1, 23)]

In [13]:
# read csv and concatenate by rows
df = pd.concat([pd.read_csv(f, sep=' ', header=0) for f in sqtl_f])

In [15]:
# keep only ones with 'UP' or 'PR' in phenotype_id
df = df[df['phenotype_id'].str.contains('UP|PR')]

In [17]:
df.columns
df.head()

Index(['phenotype_id', 'phenotype_chr', 'phenotype_start', 'phenotype_end',
       'phenotype_strand', 'num_variants', 'best_nom_dist', 'best_genotype_id',
       'best_genotype_chr', 'best_genotype_start', 'best_genotype_end',
       'dof_true', 'dof_est', 'beta_ml1', 'beta_ml2', 'pval_nom', 'pval_r2',
       'slope', 'pval_emp', 'pval_adj', 'q'],
      dtype='object')

,phenotype_id,phenotype_chr,phenotype_start,phenotype_end,phenotype_strand,num_variants,best_nom_dist,best_genotype_id,best_genotype_chr,best_genotype_start,...,dof_true,dof_est,beta_ml1,beta_ml2,pval_nom,pval_r2,slope,pval_emp,pval_adj,q
23,chr1:924948:925921:clu_5_+:PR,chr1,924949,925921,+,553,2268,chr1_928189_C_G_b38,chr1,928189,...,206,161.953,1.033240,53.6981,0.024644,0.024268,0.464043,0.915085,0.917363,0.95362
24,chr1:925189:925921:clu_5_+:PR,chr1,925190,925921,+,553,16414,chr1_942335_C_G_b38,chr1,942335,...,206,165.504,0.971104,55.3596,0.041848,0.019951,-0.361153,0.983017,0.981196,0.95695
25,chr1:925800:925921:clu_5_+:PR,chr1,925801,925921,+,549,15198,chr1_941119_A_G_b38,chr1,941119,...,206,161.984,0.842091,46.1344,0.019982,0.025999,-0.554059,0.876124,0.878662,0.94872
26,chr1:930336:931038:clu_6_+:PR,chr1,930337,931038,+,551,-82736,chr1_847601_C_T_b38,chr1,847601,...,206,173.310,1.044200,66.1787,0.002998,0.041960,0.379029,0.328671,0.329554,0.79785
27,chr1:931089:935771:clu_6_+:PR,chr1,931090,935771,+,553,-83489,chr1_847601_C_T_b38,chr1,847601,...,206,177.534,1.076380,69.4543,0.000948,0.051777,-0.419526,0.109890,0.116087,0.59994


In [23]:
intron_snp_df =df[['best_genotype_id', 'phenotype_id', 'slope', 'pval_nom', 'dof_true']]

In [24]:
intron_snp_df

,best_genotype_id,phenotype_id,slope,pval_nom,dof_true
23,chr1_928189_C_G_b38,chr1:924948:925921:clu_5_+:PR,0.464043,0.024644,206
24,chr1_942335_C_G_b38,chr1:925189:925921:clu_5_+:PR,-0.361153,0.041848,206
25,chr1_941119_A_G_b38,chr1:925800:925921:clu_5_+:PR,-0.554059,0.019982,206
26,chr1_847601_C_T_b38,chr1:930336:931038:clu_6_+:PR,0.379029,0.002998,206
27,chr1_847601_C_T_b38,chr1:931089:935771:clu_6_+:PR,-0.419526,0.000948,206
...,...,...,...,...,...
2023,chr22_50777721_C_A_b38,chr22:50782347:50783500:clu_34485_-:PR,0.522102,0.000031,206
2024,chr22_50756613_A_G_b38,chr22:50782351:50783038:clu_34485_-:PR,0.481564,0.018997,206
2025,chr22_50718238_C_T_b38,chr22:50782351:50783044:clu_34485_-:PR,-0.350582,0.003728,206
2026,chr22_50745507_C_T_b38,chr22:50782351:50783500:clu_34485_-:PR,0.559664,0.000222,206


In [25]:
import numpy as np
from scipy import stats

def compute_t_statistic(slope, p_value, df):
    """
    Compute t-statistic given slope (beta), p-value, and degrees of freedom.
    
    Parameters:
    slope (float): The slope (beta) coefficient from the regression
    p_value (float): The p-value associated with the slope
    df (int): Degrees of freedom
    
    Returns:
    float: The computed t-statistic
    """
    # Calculate the two-tailed critical value
    critical_value = stats.t.ppf((1 + (1 - p_value)) / 2, df)
    
    # Calculate the t-statistic
    t_statistic = np.sign(slope) * abs(critical_value)
    
    return t_statistic

In [33]:
compute_t_statistic(-.361, .041848, 206)

-2.047803894192153

In [34]:
stats.t.sf(2.04, 206) * 2

0.042626580676012896